In [1]:
import pprint
import numpy as np
from PIL import Image

from giza.agents import GizaAgent, AgentResult

# Make sure to fill these in
MODEL_ID = 651
VERSION_ID = 2
# As we are executing in sepolia, we need to specify the chain
CHAIN = "ethereum:sepolia:geth"
# The address of the deployed contract
MNIST_CONTRACT = "0x17807a00bE76716B91d5ba1232dd1647c4414912"

c:\Users\tyagi\Desktop\Work\Giza\giza\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# This function is for the previous MNIST tutorial
def preprocess_image(image_path: str):
    """
    Preprocess an image for the MNIST model.

    Args:
        image_path (str): Path to the image file.

    Returns:
        np.ndarray: Preprocessed image.
    """

    # Load image, convert to grayscale, resize and normalize
    image = Image.open(image_path).convert('L')
    # Resize to match the input size of the model
    image = image.resize((14, 14))
    image = np.array(image).astype('float32') / 255
    image = image.reshape(1, 196)  # Reshape to (1, 196) for model input
    return image

In [3]:
def create_agent(model_id: int, version_id: int, chain: str, contract: str):
    """
    Create a Giza agent for the MNIST model with MNIST 
    """
    agent = GizaAgent(
        contracts={"mnist": contract},
        id=model_id,
        version_id=version_id,
        chain=chain,
        account="Zenith"
    )
    return agent

In [4]:
def predict(agent: GizaAgent, image: np.ndarray):
    """
    Predict the digit in an image.

    Args:
        image (np.ndarray): Image to predict.

    Returns:
        int: Predicted digit.
    """
    prediction = agent.predict(
        input_feed={"image": image}, verifiable=True, dry_run=True
    )
    return prediction

In [5]:
def get_digit(prediction: AgentResult):
    """
    Get the digit from the prediction.

    Args:
        prediction (dict): Prediction from the model.

    Returns:
        int: Predicted digit.
    """
    # This will block the executon until the prediction has generated the proof and the proof has been verified
    return int(prediction.value[0].argmax())

In [6]:
def execute_contract(agent: GizaAgent, digit: int):
    """
    Execute the MNIST contract with the predicted digit to mint a new NFT.

    Args:
        agent (GizaAgent): Giza agent.
        digit (int): Predicted digit.

    Returns:
        str: Transaction hash.
    """
    with agent.execute() as contracts:
        contract_result = contracts.mnist.mint(digit)
        print(contract_result)
    return contract_result

In [7]:
def mint_nft_with_prediction():
    # Preprocess image
    image = preprocess_image("seven.png")
    # Create Giza agent
    agent = create_agent(MODEL_ID, VERSION_ID, CHAIN, MNIST_CONTRACT)
    # Predict digit
    prediction = predict(agent, image)
    # Get digit
    digit = get_digit(prediction)
    # Execute contract
    result = execute_contract(agent, digit)
    pprint.pprint(result)

In [8]:
# DO NOT COMMIT YOUR PASSPHRASE
import os
os.environ['MY_ACCOUNT1_PASSPHRASE'] = 'a'

In [9]:
mint_nft_with_prediction()

🚀 Starting deserialization process...
✅ Deserialization completed! 🎉


Dry run enabled. Skipping verification.


INFO: Connecting to a 'tenderly' node.


ERC721InvalidSender: sender=0x0000000000000000000000000000000000000000